## 1. Calculate the cost matrix

First, define the permutation length you are interested in. Anything below 6 should be doable, but a length of 5 could already take up to 20 minutes to find a reasonable solution (depending on your laptop).

In [1]:
PERM_LENGTH = 3

Now, we define the functions needed to calculate the cost, or distance, matrix.

In [2]:
def hamming_distance(s_1, s_2):
    
    return sum((char_1 != char_2) for char_1, char_2 in zip(s_1, s_2))

def calculate_cost(p_1, p_2):
    
    p_length = len(p_1)
    
    # Initiate cost at length of permutation
    cost = p_length
    
    # Check for equality at each offset
    for offset in range(p_length):
        
        # When hamming distance is 0, we have found cost
        if hamming_distance(p_1[offset:], p_2[:p_length - offset]) == 0:
            
            cost = offset
            
            break
            
    return cost

def calculate_cost_matrix(all_perms):
    
    size = len(all_perms)
    
    C = {}
    
    for i in range(size):
        
        perm_i = all_perms[i]
        
        for j in range(size):
            
            perm_j = all_perms[j]
            
            C[perm_i, perm_j] = calculate_cost(perm_i, perm_j)
    
    return C

Create a permutations list and calculate the cost matrix.

In [3]:
import itertools

# Create list of all permutations
index_list = [str(i + 1) for i in range(PERM_LENGTH)]

permutations = [''.join(x) for x in itertools.permutations(index_list)]

# Check length permutations list
n_perm = len(permutations)
print('There are ' + str(n_perm) + ' permutations of length ' + str(PERM_LENGTH))

There are 6 permutations of length 3


In [4]:
# Calculate cost_dict
cost_dict = calculate_cost_matrix(permutations)

## 2. Create and solve PuLP model

Import the PuLP library and define the TSP model.

In [5]:
import pulp as pl

# Define PuLP model
model = pl.LpProblem('TSP', pl.LpMinimize)

# Define binary variables
x = pl.LpVariable.dicts('x', ((i, j) for i in range(n_perm) for j in range(n_perm)), cat = 'Binary')

# Define dummy variables for subtour elimination
t = pl.LpVariable.dicts('t', (i for i in range(n_perm)), lowBound = 1, upBound = n_perm - 1, cat = 'Integer')

# Set the objective function
model += pl.lpSum(cost_dict[permutations[i], permutations[j]] * x[i, j] for i in range(n_perm) for j in range(n_perm))

# Define the constraints
for i in range(n_perm):
    
    # Do not allow loops
    model += x[i, i] == 0

for i in range(n_perm):
    
    # Exactly one outgoing edge
    model += pl.lpSum(x[i, j] for j in range(n_perm)) == 1
    
    # Exactly one incoming edge
    model += pl.lpSum(x[j, i] for j in range(n_perm)) == 1
    
    for j in range(n_perm):
        # Add subtour constraints, using n_perm as large number
        if i != j and (i != 0 and j != 0):
            
            # Time must be greater if node visited later
            model += t[j] - t[i] + n_perm * (1 - x[i, j]) >= 1

Solve the PuLP model using a solver. Here we use Gurobi, but if you do not have a Gurobi license, consider using one of the other [solvers](https://coin-or.github.io/pulp/guides/how_to_configure_solvers.html) available via PuLP.

In [6]:
# Solve the problem, here Gurobi is used
status = model.solve(pl.GUROBI(timeLimit = 60))

# Display the status and the objective function value
status, pl.LpStatus[status], pl.value(model.objective)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-01-20
Set parameter TimeLimit to value 60
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 38 rows, 41 columns and 138 nonzeros
Model fingerprint: 0x0993ea02
Variable types: 0 continuous, 41 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 14.0000000
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolved: 32 rows, 35 columns, 120 nonzeros
Variable types: 0 continuous, 35 integer (30 binary)

Root relaxation: objective 6.600000e+00, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Tim

(1, 'Optimal', 8.0)

## 3. Store and analyse the solution

Store the solution (if you found one) to visualise and analyse it.

In [7]:
source_list = []
target_list = []
cost_list = []

for i in range(n_perm):
    
    source = permutations[i]
    
    for j in range(n_perm):
        
        target = permutations[j]
        
        # Store all edges with a value of 1
        if x[i, j].value() == 1.0:
            
            source_list.append(source)
            target_list.append(target)
            
            cost = cost_dict[source, target]
            cost_list.append(cost)

Storing the solution in a dataframe is the easiest way to facilitate visualisation.

In [8]:
import pandas as pd

# Store dataframe of solution
df = pd.DataFrame()

df['source'] = source_list
df['target'] = target_list
df['value'] = cost_list

df.head(10)

,source,target,value
0,123,231,1
1,132,321,1
2,213,132,1
3,231,312,1
4,312,213,2
5,321,123,2


Create a networkx graph of your solution.

In [9]:
# Store as networkx graph
import networkx as nx

G = nx.from_pandas_edgelist(df, source = 'source', target = 'target', edge_attr = 'value')

You can now visualise the solution in an interactive HTML page!

In [10]:
# Create html of network
from pyvis.network import Network

net = Network(height = 800, width = 800, notebook = True, directed = True)

net.from_nx(G)

net.show('solution.html')